<a href="https://colab.research.google.com/github/LinhaoLi/Ants-Web-Viewer/blob/master/NLP_A2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# Code to download file into Colaboratory:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

id = '1bBM1Lxt5VXOCG_mjqDyH_VsQRGg-E-UY'
downloaded = drive.CreateFile({'id':id})
downloaded.GetContentFile('test.csv')
id = '1dT-kUFSIwb5uo9AYE-1AxPpHPrkNHaAE'
downloaded = drive.CreateFile({'id':id})
downloaded.GetContentFile('train.csv')
id = '1IOaa177I-MhPDixQNldQIxgKGO4da6WM'
downloaded = drive.CreateFile({'id':id})
downloaded.GetContentFile('val.csv')

In [0]:
import pandas as pd
train_data = pd.read_csv("train.csv")
val_data = pd.read_csv("val.csv")
test_data = pd.read_csv("test.csv")

In [0]:
train_data.head(5)

,Sentence,NER
0,-docstart-,O
1,eu rejects german call to boycott british lamb .,I-ORG O I-MISC O O O I-MISC O O
2,peter blackburn,I-PER I-PER
3,brussels 1996-08-22,I-LOC O
4,the european commission said on thursday it di...,O I-ORG I-ORG O O O O O O I-MISC O O O O O I-M...


In [0]:
val_data.head(5)

,Sentence,NER
0,-docstart-,O
1,cricket - leicestershire take over at top afte...,O O I-ORG O O O O O O O O
2,london 1996-08-30,I-LOC O
3,west indian all-rounder phil simmons took four...,I-MISC I-MISC O I-PER I-PER O O O O O O O I-OR...
4,"their stay on top , though , may be short-live...",O O O O O O O O O O O O O I-ORG O I-ORG O I-OR...


In [0]:
train_x = list(train_data.Sentence)
val_x = list(val_data.Sentence)
test_x = list(test_data.Sentence)

In [0]:
train_label = list(train_data['NER'])
val_label = list(val_data['NER'])
unique_label = list(set(" ".join(train_label).split(" ")))

In [0]:
val_data.NER

0                                                      O
1                              O O I-ORG O O O O O O O O
2                                                I-LOC O
3      I-MISC I-MISC O I-PER I-PER O O O O O O O I-OR...
4      O O O O O O O O O O O O O I-ORG O I-ORG O I-OR...
                             ...                        
695                            O I-ORG I-ORG I-ORG I-ORG
696                                O O O O O O O O O O O
697                                                O O O
698                                O I-ORG I-ORG O O O O
699                                                    O
Name: NER, Length: 700, dtype: object

In [0]:
unique_label

['I-PER', 'I-ORG', 'I-LOC', 'O', 'I-MISC']

In [0]:
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer("english")

#train_x_token = []
#train_label_token = []
#val_x_token = []
#val_label_token = []
#test_x_token = []

#train_length = []
#val_length = []
#test_length = []

In [0]:
def tokenise_x(data):
  max_length = 0
  x_token = []
  for i in data:
    sentence = i.split(" ")
    words = []
    for token in sentence:
      words.append(stemmer.stem(token))
    x_token.append(words)
    if len(sentence) > max_length:
      max_length = len(sentence)
  return x_token, max_length

In [0]:
train_x_token,train_max_len=tokenise_x(train_x)
val_x_token,val_max_len=tokenise_x(val_x)
test_x_token,test_max_len=tokenise_x(test_x)

In [0]:
def tokenise_label(data):
  train_label_token=[]
  for i in data:
    sentence = i.split(" ")
    train_label_token.append(sentence)
  return train_label_token

In [0]:
train_label_token=tokenise_label(train_label)
val_label_token=tokenise_label(val_label)

In [0]:
output_token_list = [["<BOS>"] + s for s in train_label_token]
target_token_list = [s + ["<EOS>"] for s in train_label_token]

In [0]:
from gensim.models import Word2Vec
all_words = train_x_token + val_x_token + test_x_token

In [0]:
wv_sg_model = Word2Vec(sentences=all_words,size=150, window=2, min_count=1, workers=2,sg=1)

In [0]:
wv_sg_model['eu']

array([-0.01871376,  0.01752859, -0.07587983, -0.2708914 ,  0.11486228,
       -0.0176064 ,  0.13760692,  0.04167498, -0.13319191,  0.11874988,
        0.02985683,  0.01332848, -0.11960471,  0.01309241,  0.01728505,
        0.01892452,  0.11287802, -0.06651925,  0.02682116,  0.00758533,
       -0.07488692,  0.07192501, -0.2572028 ,  0.12953913, -0.01811341,
        0.23760387, -0.00420167,  0.02529952,  0.14259784, -0.06394713,
       -0.01493964,  0.06081403,  0.0332142 , -0.05917079,  0.00266739,
       -0.08583666,  0.17983054, -0.0301755 , -0.12164801,  0.08415353,
       -0.1842655 , -0.09522513, -0.17248401,  0.16707656,  0.10716993,
        0.20076968,  0.10278208, -0.1591439 , -0.05972877, -0.05838689,
        0.02675649, -0.0054599 , -0.2691536 ,  0.02848364, -0.14835404,
       -0.02038982,  0.07625918,  0.26952186, -0.06139541,  0.05911208,
        0.08758115,  0.01691484, -0.09943899,  0.00573964, -0.11862726,
        0.03715718, -0.06739341, -0.22740245,  0.03366483, -0.04

In [0]:
import re
import numpy as np
DF = {}

for tokensized_doc in all_words:
    # get each unique word in the doc - we need to know whether the word is appeared in the document
    for term in np.unique(tokensized_doc):
        try:
            DF[term] +=1
        except:
            DF[term] =1


In [0]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from collections import Counter
import math


tf_idf = {}

# total number of documents
N = len(all_words)

doc_id = 0
# get each tokenised doc
for tokensized_doc in all_words:
    # initialise counter for the doc
    counter = Counter(tokensized_doc)
    # calculate total number of words in the doc
    total_num_words = len(tokensized_doc)    

    # get each unique word in the doc
    for term in np.unique(tokensized_doc):

        #calculate Term Frequency 
        tf = counter[term]/total_num_words
        
        #calculate Document Frequency
        df = DF[term]

        # calculate Inverse Document Frequency
        idf = math.log(N/(df+1))+1

        # calculate TF-IDF
        tf_idf[doc_id, term] = tf*idf

    doc_id += 1


In [0]:
tf_idf

{(0, '-docstart-'): 3.7867733572317377,
 (1, '.'): 0.20141025902330312,
 (1, 'boycott'): 1.023769288402527,
 (1, 'british'): 0.675381042188177,
 (1, 'call'): 0.682874517832125,
 (1, 'eu'): 0.7736257552240274,
 (1, 'german'): 0.6639161270509292,
 (1, 'lamb'): 0.9467529350069775,
 (1, 'reject'): 0.9467529350069775,
 (1, 'to'): 0.31869858164379405,
 (2, 'blackburn'): 3.711082063197344,
 (2, 'peter'): 3.6340067232837145,
 (3, '1996-08-22'): 2.568193075858512,
 (3, 'brussel'): 3.236541785848771,
 (4, '.'): 0.06042307770699094,
 (4, 'advic'): 0.28402588050209326,
 (4, 'be'): 0.14658941465182368,
 (4, 'british'): 0.2026143126564531,
 (4, 'can'): 0.22037446367261196,
 (4, 'commiss'): 0.23388996727621744,
 (4, 'consum'): 0.2399673525026826,
 (4, 'cow'): 0.25348285610628807,
 (4, 'determin'): 0.2569948732948823,
 (4, 'disagre'): 0.2936152829171526,
 (4, 'diseas'): 0.2303779500876232,
 (4, 'european'): 0.20189743914908764,
 (4, 'german'): 0.19917483811527875,
 (4, 'it'): 0.1330289571191751,
 (4, 

In [0]:
import nltk
nltk.download('brown')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
from nltk.corpus import brown
from nltk import word_tokenize

[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Unzipping corpora/brown.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [0]:
# Pretagged training data
brown_tagged_sents = brown.tagged_sents()
# Import HMM module
from nltk.tag import hmm
# Setup a trainer with default(None) values
# And train with the data
trainer = hmm.HiddenMarkovModelTrainer()
trained_tagger = trainer.train_supervised(brown_tagged_sents)
print (trained_tagger)
# Prints the basic data about the tagger
tokens = all_words[1]
print(trained_tagger.tag(tokens))

<HiddenMarkovModelTagger 472 states and 56057 output symbols>
[('eu', 'AT'), ('reject', 'AT'), ('german', 'AT'), ('call', 'AT'), ('to', 'AT'), ('boycott', 'AT'), ('british', 'AT'), ('lamb', 'AT'), ('.', 'AT')]


In [0]:
trained_tagger.tag(all_words[22])

[('rare', 'JJ'),
 ('hendrix', 'AT'),
 ('song', 'AT'),
 ('draft', 'AT'),
 ('sell', 'AT'),
 ('for', 'AT'),
 ('almost', 'AT'),
 ('$', 'AT'),
 ('17,000', 'AT'),
 ('.', 'AT')]

In [0]:
all_words[2]

['peter', 'blackburn']

In [0]:
print(brown_tagged_sents)

[[('The', 'AT'), ('Fulton', 'NP-TL'), ('County', 'NN-TL'), ('Grand', 'JJ-TL'), ('Jury', 'NN-TL'), ('said', 'VBD'), ('Friday', 'NR'), ('an', 'AT'), ('investigation', 'NN'), ('of', 'IN'), ("Atlanta's", 'NP$'), ('recent', 'JJ'), ('primary', 'NN'), ('election', 'NN'), ('produced', 'VBD'), ('``', '``'), ('no', 'AT'), ('evidence', 'NN'), ("''", "''"), ('that', 'CS'), ('any', 'DTI'), ('irregularities', 'NNS'), ('took', 'VBD'), ('place', 'NN'), ('.', '.')], [('The', 'AT'), ('jury', 'NN'), ('further', 'RBR'), ('said', 'VBD'), ('in', 'IN'), ('term-end', 'NN'), ('presentments', 'NNS'), ('that', 'CS'), ('the', 'AT'), ('City', 'NN-TL'), ('Executive', 'JJ-TL'), ('Committee', 'NN-TL'), (',', ','), ('which', 'WDT'), ('had', 'HVD'), ('over-all', 'JJ'), ('charge', 'NN'), ('of', 'IN'), ('the', 'AT'), ('election', 'NN'), (',', ','), ('``', '``'), ('deserves', 'VBZ'), ('the', 'AT'), ('praise', 'NN'), ('and', 'CC'), ('thanks', 'NNS'), ('of', 'IN'), ('the', 'AT'), ('City', 'NN-TL'), ('of', 'IN-TL'), ('Atlant

In [0]:
POS_train=[]
for i in all_words:
  sentence=[]
  for word,tag in trained_tagger.tag(i):
    sentence.append(tag)
  POS_train.append(sentence)

In [0]:
POS_train

[['AT'],
 ['AT', 'AT', 'AT', 'AT', 'AT', 'AT', 'AT', 'AT', 'AT'],
 ['AT', 'AT'],
 ['AT', 'AT'],
 ['AT',
  'AT',
  'AT',
  'AT',
  'AT',
  'AT',
  'AT',
  'AT',
  'AT',
  'AT',
  'AT',
  'AT',
  'AT',
  'AT',
  'AT',
  'AT',
  'AT',
  'AT',
  'AT',
  'AT',
  'AT',
  'AT',
  'AT',
  'AT',
  'AT',
  'AT',
  'AT',
  'AT',
  'AT',
  'AT'],
 ['AT',
  'AT',
  'AT',
  'AT',
  'AT',
  'AT',
  'AT',
  'AT',
  'AT',
  'AT',
  'AT',
  'AT',
  'AT',
  'AT',
  'AT',
  'AT',
  'AT',
  'AT',
  'AT',
  'AT',
  'AT',
  'AT',
  'AT',
  'AT',
  'AT',
  'AT',
  'AT',
  'AT',
  'AT',
  'AT',
  'AT'],
 ['AT',
  'AT',
  'AT',
  'AT',
  'AT',
  'AT',
  'AT',
  'AT',
  'AT',
  'AT',
  'AT',
  'AT',
  'AT',
  'AT',
  'AT',
  'AT',
  'AT',
  'AT',
  'AT',
  'AT',
  'AT',
  'AT',
  'AT',
  'AT',
  'AT',
  'AT',
  'AT',
  'AT',
  'AT',
  'AT',
  'AT',
  'AT',
  'AT'],
 ['PPS',
  'VBD',
  'RBR',
  'AT',
  'AT',
  'AT',
  'AT',
  'AT',
  'AT',
  'AT',
  'AT',
  'AT',
  'AT',
  'AT',
  'AT',
  'AT',
  'AT',
  'AT',
  

In [0]:
import pickle
pickle.dump(POS_train,open('POS_train.dat', 'wb'))